# Curio Face Tracking

This notebook is designed for the case study "Interactive Robotics Learning: Implementing Face Tracking with Curio."

The goal is to complete the face tracking algorithm by filling in the blanks within the main function. For each placeholder, you will choose the most appropriate option from a set of three provided choices.

There is no single "correct" answer; the goal is to engage with the task and deepen your understanding of the face tracking algorithm.

Please do not modify any other cells in the notebook.

## Imports and Initialization

In [ ]:
import sys
import os

# We need to have Curio Python library.
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import asyncio
import numpy as np
from curio import Curio
import cv2
from IPython.display import display, Image, clear_output
from threading import Event, Thread
import time
import ipywidgets as widgets

In [ ]:
mac_address = "XX:XX:XX:XX:XX:XX" # Curio MAC address
camera_ip = "XXX.XXX.XXX.XXX" # IP address of the smartphone camera
camera_port = 4747 # Port of the smartphone camera

commands = {
    'forward': b"go(50, 50, 1000);\n",
    'back': b"go(-50, -50, 1000);\n",
    'right': b"go(-50, 50, 1000);\n",
    'left': b"go(50, -50, 1000);\n",
    'stop': b"go(0, 0, 0);\n"
}

model_path = 'res10_300x300_ssd_iter_140000.caffemodel'
config_path = 'deploy.prototxt'

In [ ]:
curio = Curio(mac_address, camera_ip, camera_port)

In [ ]:
net = cv2.dnn.readNetFromCaffe(config_path, model_path)
video_output = widgets.Image(format='jpeg')

## Helper Functions

In [ ]:
def blob_standard(frame):
    return cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104, 117, 123], False, False)

def blob_high_contrast(frame):
    adjusted = cv2.convertScaleAbs(frame, alpha=1.2, beta=0)
    return cv2.dnn.blobFromImage(adjusted, 1.0, (300, 300), [104, 117, 123], False, False)

def blob_high_brightness(frame):
    adjusted = cv2.convertScaleAbs(frame, alpha=0.8, beta=30)
    return cv2.dnn.blobFromImage(adjusted, 1.0, (300, 300), [104, 117, 123], False, False)

def frame_standard(frame):
    return cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

def frame_rotate_180(frame):
    return cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

def frame_mirror(frame):
    rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    return cv2.flip(rotated_frame, 1)

def detect_faces_standard(detections):
    confidence_threshold=0.5
    detected_faces = []
    for i in range(detections.shape[2]):  
        confidence = detections[0, 0, i, 2] 
        if confidence > confidence_threshold:
            detected_faces.append(detections[0, 0, i]) 
    return detected_faces

def detect_faces_high_confidence(detections):
    confidence_threshold=0.9
    detected_faces = []
    for i in range(detections.shape[2]):  
        confidence = detections[0, 0, i, 2] 
        if confidence > confidence_threshold:
            detected_faces.append(detections[0, 0, i])
    return detected_faces

def detect_faces_low_confidence(detections):
    confidence_threshold=0.1
    detected_faces = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > confidence_threshold:
            detected_faces.append(detections[0, 0, i])
    return detected_faces

def calculate_bounding_box_standard(detection, w, h):
    box = detection[3:7] * np.array([w, h, w, h])
    return box.astype("int")

def calculate_bounding_box_centered(detection, w, h):
    box = (detection[3:7] * np.array([w, h, w, h])) + np.array([10, 10, -10, -10])
    return box.astype("int")

def calculate_bounding_box_offset(detection, w, h):
    x1, y1, x2, y2 = (detection[3:7] * np.array([w, h, w, h])).astype("int")
    return [x1 - 5, y1 - 5, x2 + 5, y2 + 5]

def control_standard(face_center_x, frame_center_x):
    if face_center_x < frame_center_x - 50:
        return commands['left']
    elif face_center_x > frame_center_x + 50:
        return commands['right']
    else:
        return commands['stop']

def control_slow_response(face_center_x, frame_center_x):
    if face_center_x < frame_center_x - 100:
        return commands['left']
    elif face_center_x > frame_center_x + 100:
        return commands['right']
    else:
        return commands['stop']

def control_fast_response(face_center_x, frame_center_x):
    if face_center_x < frame_center_x - 20:
        return commands['left']
    elif face_center_x > frame_center_x + 20:
        return commands['right']
    else:
        return commands['stop']

## Main Function

In [ ]:
def display_stream(display_stop_event):
    try:
        last_command = None
        while not display_stop_event.is_set():
            frame = curio.get_frame()
            if frame is not None:
                # *** PARTICIPANTS: Choose a rotation function ***
                rotated_frame = frame_standard(frame)
                
                # *** PARTICIPANTS: Choose a blob creation function ***
                blob = blob_standard(rotated_frame)
                net.setInput(blob)
                
                detections = net.forward()
                (h, w) = rotated_frame.shape[:2]
                
                # *** PARTICIPANTS: Choose a face detection function ***
                filtered_detections = detect_faces_standard(detections)
                
                for detection in filtered_detections:
                    # *** PARTICIPANTS: Choose a bounding box calculation function ***
                    box = calculate_bounding_box_standard(detection, w, h)
                    (x1, y1, x2, y2) = box
                    
                    cv2.rectangle(rotated_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    
                    face_center_x = (x1 + x2) // 2
                    frame_center_x = rotated_frame.shape[1] // 2
                    
                    # *** PARTICIPANTS: Choose a control function ***
                    command = control_standard(face_center_x, frame_center_x)

                    if command != last_command or command != commands['stop']:
                        curio.send_command(command)
                        last_command = command
                
                _, jpeg = cv2.imencode('.jpg', rotated_frame)
                video_output.value = jpeg.tobytes()
                
            time.sleep(0.01)
    except Exception as e:
        print(f"Error displaying stream: {e}")

## Start Detection

In [ ]:
display_stop_event = Event()
display_thread = Thread(target=display_stream, args=(display_stop_event,))
display_thread.start()

## Stop Detection

In [ ]:
display_stop_event.set()

## Display Output

In [ ]:
display(video_output)

## Stop Curio

In [ ]:
curio.stop()
display_thread.join()